# Try out CNN on EEG data with labels

## Pre-processing
+ Import data.
+ Apply filters (bandpass).
+ Detect potential bad channels and replace them by interpolation.
+ Detect potential bad epochs and remove them.

## Train CNN network
+ Define network architecture
+ Split data
+ Train model

### Use mcfly for some first model testing: https://github.com/NLeSC/mcfly

## Import packages & links

In [1]:
# Import packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import mne
#%matplotlib inline

from mayavi import mlab

In [2]:
ROOT = "C:\\OneDrive - Netherlands eScience Center\\Project_ePodium\\"
PATH_CODE = ROOT + "EEG_explorer\\"
PATH_DATA = ROOT + "Data\\"
PATH_OUTPUT = ROOT + "Data\\processed\\"
PATH_METADATA = ROOT + "Data\\metadata\\"
file_labels = "metadata.xlsx"

import sys
sys.path.insert(0, PATH_CODE)

## Load pre-processed dataset
+ See notebook for preprocessing: Exploring_EEG_data_04_prepare_data_for_ML.ipynb

In [3]:
filename = PATH_OUTPUT + "EEG_data_30channels_1s_corrected.npy"
signal_collection = np.load(filename)

filename = PATH_OUTPUT + "EEG_data_30channels_1s_corrected_labels.npy"
label_collection = np.load(filename)

## Split training data
The entire dataset is split into:
+ training data (here: about 70%) which is used to train a model.
+ validation data, used to monitor the model progress and avoid overfitting.
+ testing data, meant for final check on model performance.

#### Scale/normalize data

In [4]:
data_max = signal_collection.max()
data_min = signal_collection.min()
data_mean = signal_collection.mean()

signal_collection = signal_collection - data_mean
signal_collection = signal_collection / data_max

#from sklearn.preprocessing import MinMaxScaler


In [7]:
signal_collection.min(), signal_collection.max()

(-0.8632916943089725, 0.9997598021039686)

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(signal_collection, label_collection, test_size=0.15, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=1)

In [9]:
print('Train set size:', X_train.shape[0])
print('Validation set size:', X_val.shape[0])
print('Test set size:', X_test.shape[0])

Train set size: 28237
Validation set size: 4983
Test set size: 5863


## Switch to 1-hot encoding for labels
+ We have six categories or classes. Those are best represented by a so called 1-hot encoding. This means nothing else than simply a binary 0-or-1 for every class.. 

In [10]:
from sklearn.preprocessing import LabelBinarizer
label_transform = LabelBinarizer()

y_train_binary = label_transform.fit_transform(y_train.astype(int))
y_val_binary = label_transform.fit_transform(y_val.astype(int))
y_test_binary = label_transform.fit_transform(y_test.astype(int))

In [11]:
y_val_binary[:10,:]

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1]])

In [12]:
# Show found labels:
label_transform.classes_

array([  3,   6,  13,  26,  66, 132])

Check distribution accross the 6 label categories:

In [13]:
labels = list(label_transform.classes_)
frequencies = y_train_binary.mean(axis=0)
frequencies_df = pd.DataFrame(frequencies, index=labels, columns=['frequency'])
frequencies_df

,frequency
3,0.313206
6,0.488437
13,0.038885
26,0.061515
66,0.038141
132,0.059815


### Note:
We have more data on group 2 than on group 1. And far more data for stimuli 3 than for stimuli 13 and 66 (not surprising). 

--> post on balancing datasets: https://towardsdatascience.com/handling-imbalanced-datasets-in-deep-learning-f48407a0e758

### Needs some thinking on how to balance the data set !
e.g. by frequency dependend selection rule, or by defining a suitied special loss function....

In [65]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced'
                                               ,np.unique(y_train)
                                               ,y_train)

In [66]:
class_weight

array([0.53213101, 0.34122438, 4.28612629, 2.70936481, 4.36969978,
       2.78636274])

In [14]:
class_weight = {0: 0.53213101,
               1: 0.34122438,
               2: 4.28612629,
               3: 2.70936481,
               4: 4.36969978,
               5: 2.78636274}

## Define model architecture

In [15]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

Using TensorFlow backend.


In [37]:
np.swapaxes(X_train,1,2)[:2000,:,:].shape

(2000, 501, 30)

In [19]:
# Model test 1

n_timesteps = 501
n_features = 30
n_outputs = 6

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=5, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
#model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
#model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 497, 64)           9664      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 248, 64)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 246, 64)           12352     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 244, 64)           12352     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 122, 64)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 120, 128)          24704     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 60, 128)           0         
__________

In [20]:
epochs = 10
batch_size = 32
#verbose = 2

# fit network
model.fit(1000*np.swapaxes(X_train,1,2), y_train_binary, 
          validation_data=(1000*np.swapaxes(X_val,1,2), y_val_binary), epochs=epochs, batch_size=batch_size,
         class_weight = class_weight)


Train on 28237 samples, validate on 4983 samples
Epoch 1/10
28237/28237 [==============================] - 177s 6ms/step - loss: 13.4336 - acc: 0.3138 - val_loss: 11.0559 - val_acc: 0.3141
Epoch 2/10
28237/28237 [==============================] - 167s 6ms/step - loss: 13.4344 - acc: 0.3113 - val_loss: 11.0559 - val_acc: 0.3141
Epoch 3/10
28237/28237 [==============================] - 169s 6ms/step - loss: 13.4494 - acc: 0.3084 - val_loss: 11.0559 - val_acc: 0.3141
Epoch 4/10
28237/28237 [==============================] - 166s 6ms/step - loss: 13.4251 - acc: 0.3107 - val_loss: 11.0559 - val_acc: 0.3141
Epoch 5/10
28237/28237 [==============================] - 174s 6ms/step - loss: 13.4389 - acc: 0.3103 - val_loss: 11.0559 - val_acc: 0.3141
Epoch 6/10
28237/28237 [==============================] - 173s 6ms/step - loss: 13.4332 - acc: 0.3109 - val_loss: 11.0559 - val_acc: 0.3141
Epoch 7/10
28237/28237 [==============================] - 166s 6ms/step - loss: 13.4348 - acc: 0.3102 - val_los

## Not (yet) working...

In [100]:
# evaluate the model
#_, train_acc = model.evaluate(np.swapaxes(X_train,1,2), y_train_binary, verbose=0)
_, test_acc = model.evaluate(1000*np.swapaxes(X_test,1,2), y_test_binary, verbose=0)

In [101]:
print(test_acc)

0.8333333729985996


In [21]:
Xtest = 1000*np.swapaxes(X_test,1,2)[:200,:,:]

In [24]:
ytest = model.predict_proba(Xtest)
ytest

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]], dtype=float32)

In [23]:
model.predict(Xtest[:20,:,:])

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]], dtype=float32)